# $$ \textbf{Quantum Approximate Optimization Algorithm} \\ \textbf{Application to the Max-Cut problem} \\ \tiny \text{  Simulation and experiment on a real quantum computer}$$

# Packages and prerequisites

In [ ]:
import networkx as nx                                       # Package for creation and use of graphs


from qiskit.visualization import plot_histogram             # Quantum computing python packages
from qiskit import IBMQ, Aer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, transpile
from qiskit.tools.monitor import job_monitor


from scipy.optimize import minimize                  # Package used to minimize functions

from tqdm import tqdm                                # Package used to show loop progression


import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d



# /!\ If any change of the matplotlib method, restart the kernel 
# /!\ %matplotlib inline mandatory on the IBM website

%matplotlib qt
# To plot in a new window

# %matplotlib inline
# To plot in the notebook


import numpy as np

In [ ]:
# Saving IBM account for quantum experiments
API_KEY=''
# API_KEY is a string containing the token from IBM to use the quantum processors
IBMQ.save_account(API_KEY, overwrite=True)

# Loading our saved IBMQ accounts
IBMQ.load_account()


# If problems : 

#IBMQ.delete_account() to remove the account
#IBMQ.active_account() to check actives accounts (after they are loaded with IBMQ.load_account())
# and generate a new token from the IBM Quantum Lab Website

# Functions definitions

### Graph creation

In [ ]:
def random_circular_graph(n):                              # Function to create a n-nodes n-edges 
                                                           # random (circular) graph

    G = nx.Graph()                                         # Creation of the graph
    
    for i in range(0,n):                                   # Add n nodes to the graph
        G.add_node(i)
    
    len_nodes=len(list(G.nodes))                           
    
    for i in range(len_nodes - 1):                         # Add edges between two nodes following nodes
        G.add_edges_from([(i, i+1)])
        
    G.add_edges_from([(list(G.nodes)[-1],list(G.nodes)[0])]) 
    # Add the last edge between the last node and the first one
    
    
    return G                                              # Return the graph created

### QAOA quantum circuit creation

In [ ]:
def quantum_circuit(G,angles):                            # Function to create a quantum circuit with a graph G
                                                          # and angles (1-D array of 2p elements) as inputs
    
# Angles values must be between 0 and pi because RZZ(beta) and RX(gamma) are pi-periodic
    
    # angles is a list having the following pattern : 
    # angles = [beta_1, gamma_1, beta_2, gamma_2, ..., beta_p, gamma_p] 
    # where p is the number of times the RZZ and RX gates are applied to the qubits.
    # So we decide the value of p when calling the function to create a quantum circuit.

    
    if len(angles) % 2 != 0:
        print("Error : angles must be a 2n elements list")
        
    else:
    
        qubit_number = len(G.nodes)                           # The number of qubits and classicals bits of the circuit
        classicalbit_number = len(G.nodes)                    # are equals to the number of nodes of the graph G

    
        qc = QuantumCircuit(qubit_number,classicalbit_number) # Creation of the quantum circuit
    
        qc.h(G.nodes)                                         # Hadamard gate applied to each qubit
    
        qc.barrier()                                          # Barrier has no effect on the circuit, just make things 
                                                              # easier to visualize on the circuit drawing

    
        p=int(len(angles)/2)
        # That means we will apply p times the RZZ and RX gates to the qubits
    
        i=0
        j=1
    
        for a in range(1,p+1):    
        
            for edge in G.edges:                              # RZZ gate applied to two qubit (each time the pair of qubit
                qc.rzz(angles[i], edge[0],edge[1])            # considered are two qubits associated with nodes linked by
                                                              # an edge on the graph G)
            
                                                              # RZZ gate depends on beta parameter
                
    
            qc.barrier()

            qc.rx(angles[j], G.nodes)                         # RX gate applied to each qubit
                                                              # RX gate depends on gamma parameter
            
            qc.barrier()    
            
            i+=2
            j+=2
        
        
    
        qc.measure(G.nodes, G.nodes)                          # Measurement of each qubit, result on the assiociated 
                                                              # classical bit  

        return qc                                             # Return the quantum circuit created

### Simulation of the quantum circuit

In [ ]:
def simulation(qc,N):                                     # Function to run the quantum circuit put as an input
                                                          # in a simulator, with N the times the circuit is runned
    
    aer_sim = Aer.get_backend('aer_simulator') 
    # The simulator give us a simulaiton on how our circuit should behave on a real quantum processor
    
    shots = N                                             # Times the circuit is runned 
    
    results = aer_sim.run(qc, shots = shots).result()
    counts = results.get_counts()                         # counts is a dictionnary with all the possibles outputs 
                                                          # associated with the frequency of this output as a result
    
    
    return counts                                         # Return counts

### Real experiment with our quantum circuit on an IBM quantum processor 
https://quantum-computing.ibm.com/jobs to manage and view the status and results of all of the jobs you have run on an IBM Quantum service. 

The notebook can also be runned directly on https://quantum-computing.ibm.com/

In [ ]:
def experiment(qc,N):
    
# Function to run the quantum circuit put as an input in a simulator, with N the times the circuit is runned

    
    # Get the least busy backend device with greater than or equal to (n+1) qubits

    provider = IBMQ.get_provider(hub='ibm-q')
    backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= len(qc.qubits)+1 and
                                   not x.configuration().simulator and x.status().operational==True))
    
    
    shots = N                                             # Times the circuit is runned 

    
    transpiled_dj_circuit = transpile(qc, backend, optimization_level=3)
    job = backend.run(transpiled_dj_circuit, shots = shots)
    job_monitor(job, interval=2)
    
    
    # Get the results of the computation
    
    results = job.result()
    answer = results.get_counts()
    # answer is a dictionnary with all the possibles outputs associated with the frequency of this output as a result
    
    
    return answer                                         # Return answer

### Function to determine the average cost of a simulation or experiment results

In [ ]:
def average_cost(sim_exp_results):                       # Function do determine the average cost
                                                         # considering one simulation or experiment results
        
        
# So average cost = [Sum on every state (state cost * its frequency)] / number of times the circuit is runned
# This way of determine the cost of a graph has no link with any Hamiltonian introduced in the report
# It's just an other method that gives the same results

    results_cost=[]                      # List used to store a state, its cost and its frequency
                                         # So each element of this list is composed of 3 elements
        

    for bits, counts in sim_exp_results.items():  
    # Loop on all the state that appear in the simulation or experiment results 
    
        G = nx.Graph()                   # For each state in this list   NOT always 4, it adapts, but need to change 
                                         # this we create here a 4-nodes graph, where each node as the value of one 
                                         # qubit of this state
                
        len_bits= len(bits)              # length of a state (example : length of '01' is 2, lenght of '1101' is 4)
        
        
        for i in range (0, len_bits):    # For each bit of the bit string, 
                                         # we add a node with value the considered bit (0 or 1) 
    
            G.add_node(i, bit=bits[-(i+1)])
                 
            
        len_nodes=len(list(G.nodes))
        
        for i in range(len_nodes - 1):   # Add edges between two nodes following nodes
            G.add_edges_from([(i, i+1)])
            
        G.add_edges_from([(list(G.nodes)[-1],list(G.nodes)[0])])
        # Add the last edge between the last node and the first one

        
        for edges in G.edges:            # Loop on all the edges of the graph
            
            if G.nodes[edges[0]]["bit"]==G.nodes[edges[1]]["bit"]:  # If an edge is linked by two nodes that have the
                G.add_edge(edges[0], edges[1], weight=+1)           # same bit value, it is assigned the weight +1,
                                                                    # else the weight -1
            else:
                G.add_edge(edges[0], edges[1], weight=-1) 
            
            
# The goal of this loop is to determine the cost of each state thanks to the creation of en associated graph
        
        sum_edges_weight=0
                                            # Sum of each edge weight to have the state cost
        for edges in G.edges:                                        
            sum_edges_weight+=G.get_edge_data(edges[0],edges[1])['weight']
                
        
        results_cost.append([bits,counts,sum_edges_weight])
        # We store the cost in the results_cost list, with the state and its frequency
        
        
        
    average_cost=0                          # Here we simply calculate the average cost and check the total counts
    total_counts=0
    for a in results_cost:
        average_cost += a[1]*a[2]
        total_counts +=a[1]

    average_cost=average_cost/total_counts
    
    
    return average_cost                    # Return the calculated average cost of the simulation or experiment

### QAOA : combine all the functions in a single one
Note that when calling the function, if you want a $\textbf{circuit other than with 4 qubits}$, or if you you want to $\textbf{run the circuit other than 10 000 times}$ (for the simulation or experiment), you $\textbf{have to change these parameters}$ in the definition of the $\text{apply_qaoa()}$ function here.

In [ ]:
def apply_qaoa(x, exp=True, show=False):             # Functions that combines all the functions created above, 
                                                     # with x an 1D-array of 2p elements as an input
        
    # x is a list having the following pattern : 
    # x = [beta_1, gamma_1, beta_2, gamma_2, ..., beta_p, gamma_p] 
    # where p is the number of times the RZZ and RX gates are applied to the qubits on the quantum circuit.
    
# If exp is set to True, QAOA calls the experiment function to run the circuit on a real quantum processor
# If exp is set to False, QAOA calls the simulation function to run the circuit on a simulator

# If show is set to True, the histogram of the simulation or experiment results will be printed
        
# QAOA = Quantum Approximate Optimization Algorithm                               
    
    G = random_circular_graph(4)                  # 4-nodes 4-edges random circular graph by default
    qc = quantum_circuit(G,x)
    
    if exp == True:
        results = experiment(qc,10000)
    else:
        results = simulation(qc,10000)            # Experiment or simulation runned 10000 times by default
        
    av_c = average_cost(results)
    
    p=int(len(x)/2)
    
    if show == True and exp == False:
        plot_histogram(results, figsize=(20, 10), title='SIMULATION ; Average cost = '+str(av_c)+' ; p = '+str(p)+' ; \
QAOA parameters : '+str(x))

    if show == True and exp == True:
        plot_histogram(results, figsize=(20, 10), title='EXPERIMENT ; Average cost = '+str(av_c)+' ; p = '+str(p)+' ; \
QAOA parameters : '+str(x))
        
        
    return av_c                                  # Return the average cost calculated with the simulation

# Functions calling 
We just show here the results given by each function and then the combination of them in the $\text{apply_qaoa()}$ function.
This part is $\textbf{not mandatory to execute}$ for the next part of the notebook (QAOA parameters optimization) to work properly.

### Graph

In [ ]:
G = random_circular_graph(4)             # Creation of a graph

nx.draw(G, with_labels = True)           # Draw the graph

### Quantum circuit

In [ ]:
qc=quantum_circuit(G,[2,2])     # Create a quantum circuit with as qubit as the number of nodes in G
                                # and with beta and gamma as parameters

qc.draw('mpl')                  # Draw the quantum circuit

### Simulation

In [ ]:
sim = simulation(qc,1000)                  # Run a simulation of the quantum circuit 'qc'

print(sim)                                 # Print the count dictionnary 
plot_histogram(sim, figsize=(20,10))       # Histogram of all the outputs results and their frequency

### Average cost

In [ ]:
a_n = average_cost(sim)                 # Calculate the average cost of the result of the previous simulation

print("Average cost is", a_n)           # Printing the calculated value

### QAOA
Note that when calling the function, if you want a $\textbf{circuit other than with 4 qubits}$, or if you you want to $\textbf{run the circuit other than 10 000 times}$ (for the simulation or experiment), you $\textbf{have to change these parameters}$ in the definition of the $\text{apply_qaoa()}$ function above.

In [ ]:
value=apply_qaoa([2,2,2,2],exp=False, show=True)

print("Average cost is", value)

# QAOA parameters optimization


We use a minimizing function from the Scipy module that will try to minimize the result of the $\text{apply_qaoa()}$ function, then give us the input parameter (the 1D-array that are values for the RZZ and RX gates on our quantum circuit) that give this minimal value.

Note that when calling the function, if you want a $\textbf{circuit other than with 4 qubits}$, or if you you want to $\textbf{run the circuit other than 10 000 times}$ (for the simulation or experiment), you $\textbf{have to change these parameters}$ in the definition of the $\text{apply_qaoa()}$ function above.


### Optimizer

In [ ]:
exp = False                      # Here we decide if we want to run the optimizer with the simulator 
                                 # or on a real quantum processor
    
show = False                     # show should always be False


# Starting values for the optimizer :

start=[2, 2, 2, 2, 2, 2]     # or [1, 2.5] or [1.5, 1.5] or [2, 3] for example 


for i in range (0,len(start)):          # Starting values must be between 0 and pi because 
                                        # RZZ(beta) and RX(gamma) are pi-periodic
    if not 0 <= start[i] <= np.pi:
        print ('ERROR : starting values must be between 0 and pi')

        
p = int(len(start)/2)

    # start is a (1-D array of 2p elements) having the following pattern : 
    # start = [beta_1, gamma_1, beta_2, gamma_2, ..., beta_p, gamma_p] 
    # where p is the number of times the RZZ and RX gates are applied to the qubits.
    # So we decide the value of p when choosing a starting point.
    
    
list_of_points_av_c  = []
list_of_points_angles = []

# callbackFunction is used to store the QAOA parameters values used
# during each of its iterations by the optimizer and the average cost associated to these values

def callbackFunction(point):
    
    list_of_points_angles.append(point)
    
    a_c = apply_qaoa(point,exp=False, show=False)
    list_of_points_av_c.append(a_c) 
    

# Applying the minimization function from Scipy to the 'apply_qaoa' function
    
minimization_results = minimize(fun = apply_qaoa, 
                                method='Nelder-Mead',
                                args = (exp, show),
                                options={'fatol':0.02},                   # Convergence criteria for the average cost
                                
                                bounds = [(0, np.pi), (0, np.pi)]*p,      # Searching optimal values between 0 and pi
                                                                          # because RZZ(beta) and RX(gamma) are 
                                x0= start,                                # pi-periodic
                                callback = callbackFunction
                               )

print(minimization_results) 

# x is the best values found by the optimizer, fun is the average cost value associated to these parameters

### Results
Showing the results of the $\text{apply_qaoa()}$ function with first the starting values set previously and then with the best parameters found by the minimizer in order to have the lowest average cost possible.

In [ ]:
apply_qaoa(start,exp = False, show = True)

In [ ]:
apply_qaoa(minimization_results.x,exp = False, show = True)

# Graphic representation of the average cost and path of the minimizer 

Only for a quantum circuit where $p=1$, meaning that RZZ and RX gates are applied only one time on all the qubits.

This will give us a 3D plot of the average cost depending on the two paramaters.
With $p \ge 2$, we would have an average  cost depending on four parameters, meaning a 5D plot, which is not possible.

### Computing average cost values for different $\alpha_1$  and $\gamma_1$ values

In [ ]:
if len(minimization_results.x)==2:

    num = 50
    
    # Plot for beta and gamma between 0 and pi because RZZ(beta) and RX(gamma) are pi-periodic
    beta = np.linspace(0,np.pi,num) 
    gamma = np.linspace(0,np.pi,num)

    beta_list=[]
    gamma_list=[]
    c_list=[]

    for i in tqdm(range(0, len(beta))):
        for j in range(0, len(gamma)):
            c = apply_qaoa([beta[i],gamma[j]], exp=False, show=False)
        
            # We use here QAOA on a simulator, 
            # otherwise it would take too much time on the IBM platform
        
            beta_list.append(beta[i])
            gamma_list.append(gamma[j])
            c_list.append(c)

### 2D Representation

In [ ]:
if len(minimization_results.x)==2:

    p=int(len(minimization_results.x)/2)

    grid = tuple(zip(beta_list,gamma_list,c_list))

    fig = plt.figure(figsize =(30, 30))
    plt.tricontourf(beta_list,gamma_list,c_list)
    plt.plot(list(zip(*list_of_points_angles))[0],list(zip(*list_of_points_angles))[1], '-',color='white', 
             label='QAOA parameters values taken by the optimizer')
    plt.plot(start[0],start[1], 'o',color='white', label='Starting values of the optimizer')
    plt.plot(minimization_results.x[0],minimization_results.x[1],'o',color='red',
            label='Final values found by the optimizer')
    
    cbar = plt.colorbar()
    cbar.set_label('Average cost')
    plt.xlabel(r'$\beta_1$ (rad)')
    plt.ylabel(r'$\gamma_1$ (rad)')
    plt.title('Average cost and path taken by the optimizer'+' ; p = '+str(p))

    plt.legend()
    plt.show()

### 3D representation

In [ ]:
if len(minimization_results.x)==2:
    
    p=int(len(minimization_results.x)/2)

    fig = plt.figure(figsize =(30, 30))
    ax = fig.add_subplot(projection='3d')

    # Creating color map
    my_cmap = plt.get_cmap('Wistia')
   
    # Creating plot
    trisurf = ax.plot_trisurf(beta_list, gamma_list, c_list,
                         cmap = my_cmap,
                         linewidth = 0.02,
                         antialiased = True,
                         edgecolor = 'grey', alpha=0.4) 
    plt.xlabel(r'$\beta_1$ (rad)')
    plt.ylabel(r'$\gamma_1$ (rad)')
    
    
    ax.set_title('Average cost and path taken by the optimizer'+' ; p = '+str(p))
    
    plt.plot(minimization_results.x[0],minimization_results.x[1],minimization_results.fun,'o',color='black',
            label='Final values found by the optimizer')

    ax.plot(list(zip(*list_of_points_angles))[0], list(zip(*list_of_points_angles))[1], list_of_points_av_c, 'black',
           label='QAOA parameters values taken by the optimizer')
    
    plt.legend()

# Graphic representation QAOA parameters and average cost associated to these parameters

In [ ]:
p=int(len(minimization_results.x)/2)

fig = plt.figure(figsize =(30, 20))

plt.subplot(211)
plt.xlabel('Iterations')
plt.ylabel('radian')
plt.title('QAOA parameters values taken by the optimizer'+' ; p = '+str(p))

j=1

for i in range (0, len(minimization_results.x), 2):
    
    plt.plot(list(zip(*list_of_points_angles))[i], label = r'$\beta_{}$'.format(j))
    plt.plot(list(zip(*list_of_points_angles))[i+1], label = r'$\gamma_{}$'.format(j))
    j+=1
plt.legend()    
    
plt.subplot(212)

plt.plot(list_of_points_av_c)
plt.xlabel('Iterations')
plt.ylabel('Average cost')
plt.title('Average cost associated to these parameters')

plt.show()